# Few-Shot Prompts and Example Selectors

• Find the  [Notebook](https://colab.research.google.com/github/towardsai/ragbook-notebooks/blob/main/notebooks/Chapter%2006%20-%20Getting_the_Best_of_Few_Shot_Prompts_and_Example_Selectors.ipynb)  for this section at  [towardsai.net/book](http://towardsai.net/book).

We’ll cover how few-shot prompts and example selectors can enhance the performance of language models in LangChain. While various methods can be used to implement  **few-shot prompting**  and  **example selectors**  in LangChain, we’ll discuss three distinct approaches, examining their advantages and disadvantages.

In [ ]:
import os
from langchain_custom_utils.helper import get_openai_api_key, get_deeplake_api_key, print_response
OPENAI_API_KEY = get_openai_api_key()
DEEPLAKE_API_KEY = get_deeplake_api_key()

## Alternating Human/AI Messages

Using few-shot prompting with alternating human and AI messages is particularly useful for chat-based applications. This technique requires the language model to understand the conversational context and respond appropriately.

Although this strategy is effective in managing conversational contexts and straightforward to implement, its flexibility is limited to chat-based applications. Despite this, alternating human/AI messages can be creatively employed. In this approach, you are essentially writing the chatbot’s responses in your own words and using them as input for the model.

For example, we can create a chat prompt that translates English into pirate language by showing an example to the model using `AIMessagePromptTemplate`. Below is a code snippet illustrating it:

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain import LLMChain
from langchain.prompts.chat import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    AIMessagePromptTemplate,
    HumanMessagePromptTemplate,
)

chat = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)

template="You are a helpful assistant that translates english to pirate."
system_message_prompt = SystemMessagePromptTemplate.from_template(template)
example_human = HumanMessagePromptTemplate.from_template("Hi")
example_ai = AIMessagePromptTemplate.from_template("Argh me mateys")
human_template="{text}"
human_message_prompt = HumanMessagePromptTemplate.from_template(human_template)

chat_prompt = ChatPromptTemplate.from_messages([system_message_prompt, example_human, example_ai, human_message_prompt])
chain = LLMChain(llm=chat, prompt=chat_prompt)
chain.run("I love programming.")

## Few-Shot Prompting

Few-shot prompting can improve the output quality as the model better understands the task by reviewing the examples. However, using more tokens might lead to less effective results if the examples provided are not carefully chosen or are misleading.

Implementing the few-shot learning technique involves using the `FewShotPromptTemplate` class, which requires a `PromptTemplate` and a set of few-shot examples. The class combines the prompt template with these examples, aiding the language model in producing more accurate responses. LangChain’s `FewShotPromptTemplate` can be used to organize the approach systematically:

In [ ]:
from langchain import PromptTemplate, FewShotPromptTemplate

# create our examples
examples = [
    {
        "query": "What's the weather like?",
        "answer": "It's raining cats and dogs, better bring an umbrella!"
    }, {
        "query": "How old are you?",
        "answer": "Age is just a number, but I'm timeless."
    }
]

# create an example template
example_template = """
User: {query}
AI: {answer}
"""

# create a prompt example from above template
example_prompt = PromptTemplate(
    input_variables=["query", "answer"],
    template=example_template
)

# now break our previous prompt into a prefix and suffix
# the prefix is our instructions
prefix = """The following are excerpts from conversations with an AI
assistant. The assistant is known for its humor and wit, providing
entertaining and amusing responses to users' questions. Here are some
examples:
"""
# and the suffix our user input and output indicator
suffix = """
User: {query}
AI: """

# now create the few-shot prompt template
few_shot_prompt_template = FewShotPromptTemplate(
    examples=examples,
    example_prompt=example_prompt,
    prefix=prefix,
    suffix=suffix,
    input_variables=["query"],
    example_separator="\n\n"
)